In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import os

yolo_model_dir = '/content/yolo_model'
os.makedirs(yolo_model_dir, exist_ok=True)

with zipfile.ZipFile('/content/drive/MyDrive/Models/words-yolov5/words-yolo.zip', 'r') as f:
  f.extractall(yolo_model_dir)

In [ ]:
import json
import shutil

SOURCE2_DIR = '/content/SOURCE2/'
SOURCE3_DIR = '/content/SOURCE3/'

os.makedirs(SOURCE2_DIR, exist_ok=True)
os.makedirs(SOURCE3_DIR, exist_ok=True)

with zipfile.ZipFile('/content/drive/MyDrive/Datasets/SOURCE2/detection.zip', 'r') as zf:
  zf.extractall(SOURCE2_DIR)

with zipfile.ZipFile('/content/drive/MyDrive/Datasets/SOURCE3/detection.zip', 'r') as zf:
  zf.extractall(SOURCE3_DIR)

In [ ]:
import numpy as np
import random

source3_test_pages = [os.path.join(SOURCE3_DIR, 'test', f) for f in os.listdir(os.path.join(SOURCE3_DIR, 'test'))]

source3_test_annotations = os.path.join(SOURCE3_DIR, 'test_annotations.json')

source2_test_pages = [os.path.join(SOURCE2_DIR, 'test', f) for f in os.listdir(os.path.join(SOURCE2_DIR, 'test'))]

source2_test_annotations = os.path.join(SOURCE2_DIR, 'test_annotations.json')

test_pages = source3_test_pages + source2_test_pages
random.shuffle(test_pages)

In [ ]:
def get_source2_transcriptions(img_id, json_path):
    with open(json_path, 'r') as f:
        contents = json.loads(f.read())

    words = contents[img_id]["words"]

    transcriptions = [w['transcription'] for w in words]

    return " ".join(transcriptions)

def get_source3_transcriptions(img_id, json_path):
    with open(json_path, 'r') as f:
        contents = json.loads(f.read())

    regions_contents = contents[img_id]["regions_contents"]

    transcriptions = []
    for rc in regions_contents:
        transcriptions += rc['transcriptions']

    return " ".join(transcriptions)

def parse_transcriptions(pages, source2_annotations, source3_annotations, annotations):
    for page_path in pages:
        page_id = os.path.basename(page_path[:-4])
        if SOURCE2_DIR in page_path:
            transcriptions = get_source2_transcriptions(page_id, source2_annotations)
            if page_id not in [*annotations]:
                annotations[page_id] = {"transcriptions": transcriptions}
            else:
                raise ValueError(f"Page with id: {page_id} is already in annotations")

        elif SOURCE3_DIR in page_path:
            transcriptions = get_source3_transcriptions(page_id, source3_annotations)
            if page_id not in [*annotations]:
                annotations[page_id] = {"transcriptions": transcriptions}
            else:
                raise ValueError(f"Page with id: {page_id} is already in annotations")

In [ ]:
test_annotations = {}
parse_transcriptions(test_pages, source2_test_annotations,
                  source3_test_annotations, test_annotations)

In [ ]:
def group_boxes_by_row(boxes, y_margin_ratio=0.35):
    sorted_boxes = sorted(boxes, key=lambda box: int(box.xyxy[0][1].cpu().numpy()))

    rows = []
    current_row = []
    prev_y = None

    for box in sorted_boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        box_height = y2 - y1

        y_threshold = box_height * y_margin_ratio

        if prev_y is None or abs(y1 - prev_y) > y_threshold:
            if current_row:
                rows.append(current_row)
            current_row = [box]
        else:
            current_row.append(box)

        prev_y = y1

    if current_row:
        rows.append(current_row)

    for row in rows:
        row.sort(key=lambda box: int(box.xyxy[0][0].cpu().numpy()))

    return [box for row in rows for box in row]


In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np
import shutil
import time

font_path = "/content/drive/MyDrive/etc/Roboto-Bold.ttf"
font_size = 24

font = ImageFont.truetype(font_path, font_size)
color = (128, 0, 128)

yolo_model = YOLO('/content/yolo_model/best.pt')
yolo_model.overrides['data'] = '/content/yolo_model/yolo_config.yaml'

eval_dir = "/content/eval_results/"
os.makedirs(eval_dir, exist_ok=True)
for img_path in test_pages[:50]:
    img = Image.open(img_path).convert('L').convert('RGB')
    img_np = np.array(img)

    results = yolo_model.predict(source=img_np, imgsz=946, device='cpu', max_det=300)


    results = results[0]
    sorted_boxes = group_boxes_by_row(results.boxes)

    draw = ImageDraw.Draw(img)

    for i, box in enumerate(sorted_boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        draw.text((x1, y1), str(i + 1), fill=color, font=font)

    annotated_image_path = os.path.join(eval_dir, os.path.basename(img_path))
    img.save(annotated_image_path)



In [ ]:
import shutil
shutil.make_archive('/content/eval_results' ,'zip', '/content/eval_results')